
### Metastore level storage

- Metastore is attached to the data lake.  
- Catalog is managed.
- Schema is managed.
- Object is managed.

Now, data for our objects [managed tables] will be stored in the Metastore Data Lake

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS managed_catalog;

create schema if not exists managed_catalog.managed_schema;

create table if not exists managed_catalog.managed_schema.managed_table 
( 
id int,
name string
);

Insert into managed_catalog.managed_schema.managed_table values (1,'John'),(2, "mary"), (3, "mike");

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from managed_catalog.managed_schema.managed_table;

id,name
1,John
2,mary
3,mike



### Catalog level storage

- Metastore is attached to the data lake.  
- Catalog is attached to the data lake.
- Schema is managed.
- Object is managed.


In [0]:
%sql
CREATE CATALOG IF NOT EXISTS external_catalog
MANAGED LOCATION 'abfss://metastore@storageunitycatalogjrt.dfs.core.windows.net/catalog';

create schema if not exists external_catalog.managed_schema_ext_cat;

create table if not exists external_catalog.managed_schema_ext_cat.managed_table 
( 
id int,
name string
);

Insert into external_catalog.managed_schema_ext_cat.managed_table values (1,'John'),(2, "mary"), (3, "mike");



num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from external_catalog.managed_schema_ext_cat.managed_table;

id,name
1,John
2,mary
3,mike



### Schema level storage

- Metastore is attached to the data lake.  
- Catalog is attached to the data lake.
- Schema is attached to the data lake.
- Object is managed.
 

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS external_catalog
MANAGED LOCATION 'abfss://metastore@storageunitycatalogjrt.dfs.core.windows.net/catalog';

create schema if not exists external_catalog.ext_schema
managed location 'abfss://metastore@storageunitycatalogjrt.dfs.core.windows.net/schema';;

create table if not exists external_catalog.ext_schema.managed_table 
( 
id int,
name string
);

Insert into external_catalog.ext_schema.managed_table values (1,'John'),(2, "mary"), (3, "mike");



num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql

select * from external_catalog.ext_schema.managed_table;

id,name
1,John
2,mary
3,mike


In [0]:
%sql

drop table external_catalog.ext_schema.managed_table;

In [0]:
%sql
undrop table external_catalog.ext_schema.managed_table


### Object level storge ( External Tables )

- Metastore is attached to the data lake.  
- Catalog is attached to the data lake.
- Schema is attached to the data lake.
- Object is attached to the data lake
 

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS external_catalog
MANAGED LOCATION 'abfss://metastore@storageunitycatalogjrt.dfs.core.windows.net/catalog';

create schema if not exists external_catalog.ext_schema
managed location 'abfss://metastore@storageunitycatalogjrt.dfs.core.windows.net/schema';

create table if not exists external_catalog.ext_schema.ext_table 
( 
id int,
name string
)
location 'abfss://raw@storageunitycatalogjrt.dfs.core.windows.net/ext_table';

Insert into external_catalog.ext_schema.ext_table values (1,'John'),(2, "mary"), (3, "mike");



num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql

select * from external_catalog.ext_schema.ext_table;

id,name
1,John
2,mary
3,mike



### Volumes ( External )

In [0]:
%sql

create external volume external_catalog.ext_schema.ext_volume
location 'abfss://raw@storageunitycatalogjrt.dfs.core.windows.net/data';

In [0]:
%sql

select * from csv.`/Volumes/external_catalog/ext_schema/ext_volume/DimAirline.csv`;

_c0,_c1,_c2
airline_id,airline_name,country
1,United Airlines,USA
2,Air India,India
3,Delta Airlines,USA
4,Qantas,Australia
5,Lufthansa,Germany
6,Air France,France
7,Emirates,UAE
8,British Airways,UK
9,Singapore Airlines,Singapore



### Data Masking ( Unity Catalog Functions)

In [0]:
%sql

create table external_catalog.ext_schema.employee
(
  id int,
  name string,
  salary int

);

 insert into external_catalog.ext_schema.employee 
 values (1, "John", 10000), (2, 'Mary',20000), (3, 'Mike',30000);


select * from external_catalog.ext_schema.employee;



id,name,salary
1,John,10000
2,Mary,20000
3,Mike,30000


In [0]:
%sql
CREATE OR REPLACE FUNCTION external_catalog.ext_schema.masking(Salary STRING)
  RETURN CASE WHEN is_account_group_member('HR') THEN salary ELSE '***-**-****' END;


In [0]:
%sql
ALTER TABLE external_catalog.ext_schema.employee ALTER COLUMN salary SET MASK external_catalog.ext_schema.masking;


In [0]:
%sql
select * from external_catalog.ext_schema.employee;

id,name,salary
1,John,10000
2,Mary,20000
3,Mike,30000
